## Accelerate Inference: Neural Network Pruning (IMPROVED)

In [1]:
import os
import numpy as np
import cv2
import pickle
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchsummary import summary

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# untar
!ls
!tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

dataset.tar.gz	sample_data
train_images.pkl
train_labels.pkl
val_images.pkl
val_labels.pkl


In [4]:
train_images = torch.tensor(train_images, dtype=torch.float32)
val_images = torch.tensor(val_images, dtype=torch.float32)

train_images = train_images.permute(0, 3, 1, 2)
val_images = val_images.permute(0, 3, 1, 2)

print(f"Train images shape: {train_images.shape}")
print(f"Val images shape: {val_images.shape}")

Train images shape: torch.Size([22475, 3, 25, 25])
Val images shape: torch.Size([2525, 3, 25, 25])


In [5]:
class AugmentedDataset(Dataset):
    def __init__(self, images, labels, is_train=True):
        self.images = images
        self.labels = labels
        self.is_train = is_train

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]

        if self.is_train:
            # Random horizontal flip
            if torch.rand(1) > 0.5:
                img = torch.flip(img, [2])  # Flip along width dimension

            # Random brightness and contrast adjustment
            if torch.rand(1) > 0.3:
                brightness_factor = 0.8 + torch.rand(1).item() * 0.4  # [0.8, 1.2]
                img = img * brightness_factor
                img = torch.clamp(img, 0, 255)

            # Random noise
            if torch.rand(1) > 0.5:
                noise = torch.randn_like(img) * 2.0
                img = img + noise
                img = torch.clamp(img, 0, 255)

        return img, label

# Create augmented datasets
train_dataset = AugmentedDataset(
    train_images,
    torch.tensor(train_labels.squeeze(), dtype=torch.long),
    is_train=True
)
val_dataset = AugmentedDataset(
    val_images,
    torch.tensor(val_labels.squeeze(), dtype=torch.long),
    is_train=False
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")

Train dataset size: 22475
Val dataset size: 2525


In [6]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [7]:
def create_model_with_bn(cfg=None):
    if cfg is None:
        cfg = [32, 32, 64, 64]

    model = nn.Sequential(
        nn.Conv2d(3, cfg[0], kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(cfg[0]),
        nn.ReLU(),
        nn.Conv2d(cfg[0], cfg[1], kernel_size=3, padding=0, bias=False),
        nn.BatchNorm2d(cfg[1]),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Dropout(0.25),

        nn.Conv2d(cfg[1], cfg[2], kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(cfg[2]),
        nn.ReLU(),
        nn.Conv2d(cfg[2], cfg[3], kernel_size=3, padding=0, bias=False),
        nn.BatchNorm2d(cfg[3]),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Dropout(0.25),

        nn.Flatten(),
        nn.Linear(cfg[3] * 4 * 4, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 5),
    )
    return model

In [8]:
def train_one_epoch_with_bn_reg(model, train_loader, optimizer, criterion, device, l1_lambda=0.0):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    train_loader_tqdm = tqdm(train_loader, desc="Training", leave=False)

    for inputs, labels in train_loader_tqdm:
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        if l1_lambda > 0:
            bn_regularization = 0.0
            for module in model.modules():
                if isinstance(module, nn.BatchNorm2d):
                    bn_regularization += torch.sum(torch.abs(module.weight))
            loss = loss + l1_lambda * bn_regularization

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        train_loader_tqdm.set_postfix(loss=running_loss / total, accuracy=100 * correct / total)

    return running_loss / len(train_loader), 100 * correct / total

In [9]:
def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    val_loader_tqdm = tqdm(val_loader, desc="Validation", leave=False)

    with torch.no_grad():
        for inputs, labels in val_loader_tqdm:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            val_loader_tqdm.set_postfix(loss=val_loss / total, accuracy=100 * correct / total)

    return val_loss / len(val_loader), 100 * correct / total

In [10]:
def gather_bn_weights(model):
    bn_weights = []
    for module in model.modules():
        if isinstance(module, nn.BatchNorm2d):
            bn_weights.append(module.weight.data.abs().clone())
    return torch.cat(bn_weights)


def compute_pruning_threshold(model, percent):
    """
    Enhanced adaptive threshold selection
    """
    all_bn_weights = gather_bn_weights(model)
    sorted_bn = torch.sort(all_bn_weights)[0]

    threshold_index = int(len(sorted_bn) * percent / 100)
    percentile_threshold = sorted_bn[threshold_index]

    mean_weight = all_bn_weights.mean()
    std_weight = all_bn_weights.std()

    if abs(percentile_threshold - mean_weight) < 0.5 * std_weight:
        adjusted_threshold = mean_weight - (percent / 100.0) * std_weight
        threshold = max(adjusted_threshold, sorted_bn[min(threshold_index, len(sorted_bn)-1)])
    else:
        threshold = percentile_threshold

    return threshold


def prune_model(model, percent):
    threshold = compute_pruning_threshold(model, percent)
    print(f"Pruning threshold: {threshold:.6f}")

    cfg_mask = []
    cfg = []
    for module in model.modules():
        if isinstance(module, nn.BatchNorm2d):
            weight_copy = module.weight.data.abs().clone()
            mask = weight_copy.gt(threshold).float()
            cfg_mask.append(mask)
            cfg.append(int(torch.sum(mask)))

    conv_layers = []
    bn_layers = []
    linear_layers = []

    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            conv_layers.append(module)
        elif isinstance(module, nn.BatchNorm2d):
            bn_layers.append(module)
        elif isinstance(module, nn.Linear):
            linear_layers.append(module)

    for i, (conv, bn, mask) in enumerate(zip(conv_layers, bn_layers, cfg_mask)):
        bn.weight.data.mul_(mask)
        bn.bias.data.mul_(mask)

        for j in range(len(mask)):
            if mask[j] == 0:
                conv.weight.data[j, :, :, :] = 0

        if i + 1 < len(conv_layers):
            next_conv = conv_layers[i + 1]
            for j in range(len(mask)):
                if mask[j] == 0:
                    next_conv.weight.data[:, j, :, :] = 0

    if len(linear_layers) > 0 and len(cfg_mask) > 0:
        first_linear = linear_layers[0]
        last_mask = cfg_mask[-1]
        spatial_size = 4 * 4

        for j in range(len(last_mask)):
            if last_mask[j] == 0:
                start_idx = j * spatial_size
                end_idx = (j + 1) * spatial_size
                first_linear.weight.data[:, start_idx:end_idx] = 0

    return cfg


# IMPROVEMENT 3: Add Linear Layer Pruning
def prune_linear_layers(model, percent=50):
    """
    Prune linear layer weights by magnitude
    """
    print(f"\nPruning linear layers by {percent}%...")
    for module in model.modules():
        if isinstance(module, nn.Linear):
            weight = module.weight.data
            threshold = torch.quantile(torch.abs(weight), percent / 100.0)
            mask = torch.abs(weight) > threshold
            module.weight.data *= mask.float()

            # Also prune bias for pruned output neurons (optional)
            if module.bias is not None:
                row_sums = torch.abs(weight).sum(dim=1)
                bias_mask = row_sums > 0
                module.bias.data *= bias_mask.float()


def calculate_sparsity(model):
    """Calculate sparsity of the model."""
    num_zero = 0
    total_params = 0

    for param in model.parameters():
        total_params += param.numel()
        num_zero += torch.sum(param.data == 0).item()

    sparsity = num_zero / total_params if total_params > 0 else 0
    return sparsity, num_zero, total_params

In [11]:
# STEP 1: Train model with BN and L1 regularization
print("="*60)
print("STEP 1: Training with L1 regularization on BN")
print("="*60)

model_bn = create_model_with_bn()
model_bn = model_bn.to(device)

criterion = nn.CrossEntropyLoss()

# IMPROVEMENT 4: Extended training with warmup
num_epochs = 150  # Increased from 100
l1_lambda = 3e-4  # Slightly reduced for better initial accuracy

optimizer = torch.optim.Adam(model_bn.parameters(), lr=0.001, weight_decay=1e-6)

# Add warmup scheduler
from torch.optim.lr_scheduler import LinearLR, SequentialLR
warmup_epochs = 10
warmup_scheduler = LinearLR(optimizer, start_factor=0.1, total_iters=warmup_epochs)
main_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs-warmup_epochs)
scheduler = SequentialLR(optimizer, schedulers=[warmup_scheduler, main_scheduler], milestones=[warmup_epochs])

# IMPROVEMENT 5: Track best model
best_val_accuracy_step1 = 0
best_model_state_step1 = None

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    train_loss, train_accuracy = train_one_epoch_with_bn_reg(
        model_bn, train_loader, optimizer, criterion, device, l1_lambda
    )

    val_loss, val_accuracy = validate(model_bn, val_loader, criterion, device)

    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%, '
          f'LR: {scheduler.get_last_lr()[0]:.6f}')

    # Track best model
    if val_accuracy > best_val_accuracy_step1:
        best_val_accuracy_step1 = val_accuracy
        best_model_state_step1 = {k: v.cpu().clone() for k, v in model_bn.state_dict().items()}
        print(f"  *** New best: {best_val_accuracy_step1:.2f}% ***")

    scheduler.step()

# Load best model from step 1
if best_model_state_step1:
    print(f"\nLoading best model from training: {best_val_accuracy_step1:.2f}%")
    model_bn.load_state_dict({k: v.to(device) for k, v in best_model_state_step1.items()})

print("\nInitial model performance:")
val_loss, val_accuracy = validate(model_bn, val_loader, criterion, device)
print(f"Validation Accuracy: {val_accuracy:.2f}%")

STEP 1: Training with L1 regularization on BN
Epoch 1/150


Train Loss: 1.6685, Train Acc: 23.22%, Val Loss: 1.5119, Val Acc: 32.48%, LR: 0.000100
  *** New best: 32.48% ***
Epoch 2/150


Train Loss: 1.6082, Train Acc: 27.65%, Val Loss: 1.4153, Val Acc: 37.07%, LR: 0.000190
  *** New best: 37.07% ***
Epoch 3/150


Train Loss: 1.5751, Train Acc: 30.02%, Val Loss: 1.3783, Val Acc: 38.73%, LR: 0.000280
  *** New best: 38.73% ***
Epoch 4/150


Train Loss: 1.5561, Train Acc: 31.33%, Val Loss: 1.3176, Val Acc: 42.69%, LR: 0.000370
  *** New best: 42.69% ***
Epoch 5/150


...

Train Loss: 1.0383, Train Acc: 55.54%, Val Loss: 0.6271, Val Acc: 76.95%, LR: 0.000002
Epoch 148/150


Train Loss: 1.0390, Train Acc: 55.57%, Val Loss: 0.6287, Val Acc: 77.19%, LR: 0.000001
Epoch 149/150


Train Loss: 1.0405, Train Acc: 55.29%, Val Loss: 0.6298, Val Acc: 77.35%, LR: 0.000001
Epoch 150/150


Train Loss: 1.0241, Train Acc: 56.21%, Val Loss: 0.6274, Val Acc: 77.43%, LR: 0.000000

Loading best model from training: 78.02%

Initial model performance:


Validation Accuracy: 78.02%


In [12]:
# STEP 2: Gradual Iterative Pruning
print("\n" + "="*60)
print("STEP 2: Gradual Iterative Pruning")
print("="*60)

bn_weights = gather_bn_weights(model_bn).cpu().numpy()
print(f"BN weights - Min: {bn_weights.min():.6f}, Max: {bn_weights.max():.6f}, "
      f"Mean: {bn_weights.mean():.6f}, Median: {np.median(bn_weights):.6f}")

pruning_schedule = [25, 40, 50, 60, 65]
finetune_epochs_per_step = 30

optimizer = torch.optim.Adam(model_bn.parameters(), lr=0.0005, weight_decay=5e-4)

best_overall_model = None
best_overall_accuracy = 0

for prune_percent in pruning_schedule:
    print(f"\n{'='*60}")
    print(f"Pruning to {prune_percent}%...")
    print(f"{'='*60}")

    cfg = prune_model(model_bn, prune_percent)
    print(f"Channel configuration after {prune_percent}% pruning: {cfg}")

    sparsity, num_zero, total_params = calculate_sparsity(model_bn)
    print(f"Sparsity: {100*sparsity:.2f}% ({num_zero}/{total_params} zeros)")

    val_loss, val_accuracy = validate(model_bn, val_loader, criterion, device)
    print(f"Validation Accuracy after pruning: {val_accuracy:.2f}%")

    masks = {}
    for name, param in model_bn.named_parameters():
        masks[name] = (param.data != 0).float().to(device)

    print(f"\nFine-tuning for {finetune_epochs_per_step} epochs...")
    best_in_step = 0
    for epoch in range(finetune_epochs_per_step):
        train_loss, train_accuracy = train_one_epoch_with_bn_reg(
            model_bn, train_loader, optimizer, criterion, device, l1_lambda=0
        )

        with torch.no_grad():
            for name, param in model_bn.named_parameters():
                param.data.mul_(masks[name])

        if (epoch + 1) % 5 == 0:
            val_loss, val_accuracy = validate(model_bn, val_loader, criterion, device)
            print(f"  Epoch {epoch+1}/{finetune_epochs_per_step}: "
                  f"Train Acc: {train_accuracy:.2f}%, Val Acc: {val_accuracy:.2f}%")

            # Track best in this step
            if val_accuracy > best_in_step:
                best_in_step = val_accuracy

    val_loss, val_accuracy = validate(model_bn, val_loader, criterion, device)
    print(f"\nAfter {prune_percent}% pruning + fine-tuning: Val Acc = {val_accuracy:.2f}%")

    # Track best overall model
    if val_accuracy > best_overall_accuracy:
        best_overall_accuracy = val_accuracy
        best_overall_model = {k: v.cpu().clone() for k, v in model_bn.state_dict().items()}
        print(f"  *** New overall best: {best_overall_accuracy:.2f}% ***")

print("\n" + "="*60)
print("PRUNING RESULTS BEFORE LINEAR LAYER PRUNING")
print("="*60)
sparsity, num_zero, total_params = calculate_sparsity(model_bn)
val_loss, val_accuracy = validate(model_bn, val_loader, criterion, device)
print(f"Validation Accuracy: {val_accuracy:.2f}%")
print(f"Sparsity: {100*sparsity:.2f}% ({num_zero}/{total_params} zeros)")


STEP 2: Gradual Iterative Pruning
BN weights - Min: 0.000002, Max: 1.066803, Mean: 0.399659, Median: 0.376439

Pruning to 25%...
Pruning threshold: 0.296871
Channel configuration after 25% pruning: [22, 29, 50, 42]
Sparsity: 34.97% (207412/593125 zeros)


Validation Accuracy after pruning: 70.50%

Fine-tuning for 30 epochs...


  Epoch 5/30: Train Acc: 48.57%, Val Acc: 75.52%


  Epoch 10/30: Train Acc: 49.14%, Val Acc: 74.30%


  Epoch 15/30: Train Acc: 49.27%, Val Acc: 73.74%


...

  Epoch 20/30: Train Acc: 44.90%, Val Acc: 68.71%


  Epoch 25/30: Train Acc: 45.34%, Val Acc: 69.31%


  Epoch 30/30: Train Acc: 45.23%, Val Acc: 69.62%



After 65% pruning + fine-tuning: Val Acc = 69.62%

PRUNING RESULTS BEFORE LINEAR LAYER PRUNING


Validation Accuracy: 69.62%
Sparsity: 77.32% (458610/593125 zeros)


In [13]:
print("\n" + "="*60)
print("STEP 2.5: Linear Layer Pruning")
print("="*60)

# Prune linear layers
prune_linear_layers(model_bn, percent=60)

sparsity, num_zero, total_params = calculate_sparsity(model_bn)
print(f"Sparsity after linear pruning: {100*sparsity:.2f}% ({num_zero}/{total_params} zeros)")

val_loss, val_accuracy = validate(model_bn, val_loader, criterion, device)
print(f"Validation Accuracy after linear pruning: {val_accuracy:.2f}%")

# Update masks to include linear layer pruning
masks = {}
for name, param in model_bn.named_parameters():
    masks[name] = (param.data != 0).float().to(device)

# Quick fine-tune after linear pruning
print("\nFine-tuning after linear layer pruning (20 epochs)...")
optimizer = torch.optim.Adam(model_bn.parameters(), lr=0.0003, weight_decay=5e-4)

for epoch in range(20):
    train_loss, train_accuracy = train_one_epoch_with_bn_reg(
        model_bn, train_loader, optimizer, criterion, device, l1_lambda=0
    )

    with torch.no_grad():
        for name, param in model_bn.named_parameters():
            param.data.mul_(masks[name])

    if (epoch + 1) % 5 == 0:
        val_loss, val_accuracy = validate(model_bn, val_loader, criterion, device)
        print(f"  Epoch {epoch+1}/20: Train Acc: {train_accuracy:.2f}%, Val Acc: {val_accuracy:.2f}%")

val_loss, val_accuracy = validate(model_bn, val_loader, criterion, device)
sparsity, num_zero, total_params = calculate_sparsity(model_bn)
print(f"\nAfter linear pruning + fine-tuning:")
print(f"Validation Accuracy: {val_accuracy:.2f}%")
print(f"Sparsity: {100*sparsity:.2f}% ({num_zero}/{total_params} zeros)")


STEP 2.5: Linear Layer Pruning

Pruning linear layers by 60%...
Sparsity after linear pruning: 77.58% (460146/593125 zeros)


Validation Accuracy after linear pruning: 69.74%

Fine-tuning after linear layer pruning (20 epochs)...


  Epoch 5/20: Train Acc: 45.73%, Val Acc: 70.57%


  Epoch 10/20: Train Acc: 46.08%, Val Acc: 70.46%


  Epoch 15/20: Train Acc: 46.64%, Val Acc: 70.69%


  Epoch 20/20: Train Acc: 46.31%, Val Acc: 68.79%



After linear pruning + fine-tuning:
Validation Accuracy: 68.79%
Sparsity: 77.58% (460146/593125 zeros)


In [14]:
# STEP 3: Enhanced Final Fine-tuning
print("\n" + "="*60)
print("STEP 3: Enhanced Final Fine-tuning")
print("="*60)

num_finetune_epochs = 60
optimizer = torch.optim.Adam(model_bn.parameters(), lr=0.0005, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_finetune_epochs)

# Ensure masks are up-to-date
masks = {}
for name, param in model_bn.named_parameters():
    masks[name] = (param.data != 0).float().to(device)

best_val_accuracy = 0
best_final_model = None

for epoch in range(num_finetune_epochs):
    print(f"Final Fine-tune Epoch {epoch+1}/{num_finetune_epochs}")

    train_loss, train_accuracy = train_one_epoch_with_bn_reg(
        model_bn, train_loader, optimizer, criterion, device, l1_lambda=0
    )

    with torch.no_grad():
        for name, param in model_bn.named_parameters():
            param.data.mul_(masks[name])

    val_loss, val_accuracy = validate(model_bn, val_loader, criterion, device)

    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%, '
          f'LR: {scheduler.get_last_lr()[0]:.6f}')

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_final_model = {k: v.cpu().clone() for k, v in model_bn.state_dict().items()}
        print(f"  *** New best validation accuracy: {best_val_accuracy:.2f}% ***")

    scheduler.step()

# Load best model from final fine-tuning
if best_final_model:
    print(f"\nLoading best model from final fine-tuning: {best_val_accuracy:.2f}%")
    model_bn.load_state_dict({k: v.to(device) for k, v in best_final_model.items()})

print(f"\nBest validation accuracy during fine-tuning: {best_val_accuracy:.2f}%")


STEP 3: Enhanced Final Fine-tuning
Final Fine-tune Epoch 1/60


Train Loss: 1.2401, Train Acc: 45.85%, Val Loss: 0.7985, Val Acc: 71.01%, LR: 0.000500
  *** New best validation accuracy: 71.01% ***
Final Fine-tune Epoch 2/60


Train Loss: 1.2483, Train Acc: 45.35%, Val Loss: 0.8326, Val Acc: 67.76%, LR: 0.000500
Final Fine-tune Epoch 3/60


Train Loss: 1.2462, Train Acc: 45.14%, Val Loss: 0.8105, Val Acc: 69.94%, LR: 0.000499
Final Fine-tune Epoch 4/60


Train Loss: 1.2405, Train Acc: 45.64%, Val Loss: 0.8702, Val Acc: 66.30%, LR: 0.000497
Final Fine-tune Epoch 5/60


...

Train Loss: 1.1885, Train Acc: 48.36%, Val Loss: 0.7449, Val Acc: 72.40%, LR: 0.000009
Final Fine-tune Epoch 57/60


Train Loss: 1.1906, Train Acc: 47.84%, Val Loss: 0.7454, Val Acc: 72.79%, LR: 0.000005
Final Fine-tune Epoch 58/60


Train Loss: 1.2007, Train Acc: 47.54%, Val Loss: 0.7564, Val Acc: 72.28%, LR: 0.000003
Final Fine-tune Epoch 59/60


Train Loss: 1.1869, Train Acc: 47.72%, Val Loss: 0.7593, Val Acc: 72.40%, LR: 0.000001
Final Fine-tune Epoch 60/60


Train Loss: 1.1902, Train Acc: 48.31%, Val Loss: 0.7465, Val Acc: 72.75%, LR: 0.000000

Loading best model from final fine-tuning: 73.07%

Best validation accuracy during fine-tuning: 73.07%


In [15]:
# Transfer weights from BN model to original architecture
print("\n" + "="*60)
print("Transferring weights to original architecture")
print("="*60)

# Define original ConvNet WITHOUT BatchNorm
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1, bias=True),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=0, bias=True),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),
            nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=True),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=0, bias=True),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),
            nn.Flatten(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 5),
        )
    def forward(self, x):
        return self.model(x)

original_model = ConvNet().to(device)

bn_conv_layers = []
bn_bn_layers = []
for module in model_bn.modules():
    if isinstance(module, nn.Conv2d):
        bn_conv_layers.append(module)
    elif isinstance(module, nn.BatchNorm2d):
        bn_bn_layers.append(module)

orig_conv_layers = []
for module in original_model.modules():
    if isinstance(module, nn.Conv2d):
        orig_conv_layers.append(module)

with torch.no_grad():
    for i, (orig_conv, bn_conv, bn) in enumerate(zip(orig_conv_layers, bn_conv_layers, bn_bn_layers)):
        orig_conv.weight.data = bn_conv.weight.data.clone()

        gamma = bn.weight.data
        beta = bn.bias.data
        mean = bn.running_mean
        var = bn.running_var
        eps = bn.eps

        std = torch.sqrt(var + eps)
        bn_weight_normalized = gamma / std
        bn_bias_normalized = beta - gamma * mean / std

        for j in range(orig_conv.weight.data.shape[0]):
            orig_conv.weight.data[j] = orig_conv.weight.data[j] * bn_weight_normalized[j]

        orig_conv.bias.data = bn_bias_normalized.clone()

    bn_linear_layers = [m for m in model_bn.modules() if isinstance(m, nn.Linear)]
    orig_linear_layers = [m for m in original_model.modules() if isinstance(m, nn.Linear)]

    for bn_linear, orig_linear in zip(bn_linear_layers, orig_linear_layers):
        orig_linear.weight.data = bn_linear.weight.data.clone()
        orig_linear.bias.data = bn_linear.bias.data.clone()

print("Weights transferred successfully!")

val_loss, val_accuracy = validate(original_model, val_loader, criterion, device)
sparsity, num_zero, total_params = calculate_sparsity(original_model)

print(f"\nOriginal architecture with pruned weights:")
print(f"Validation Accuracy: {val_accuracy:.2f}%")
print(f"Sparsity: {100*sparsity:.2f}% ({num_zero}/{total_params} zeros)")

accuracy = val_accuracy / 100
if accuracy > 0.6 and sparsity > 0:
    score = (accuracy + sparsity) / 2
else:
    score = 0
print(f"FINAL SCORE: {score:.4f}")

torch.save(original_model.state_dict(), 'pruned_model_for_autograder.pt', _use_new_zipfile_serialization=False)
print("\nSaved pruned model for autograder: pruned_model_for_autograder.pt")


Transferring weights to original architecture
Weights transferred successfully!



Original architecture with pruned weights:
Validation Accuracy: 73.07%
Sparsity: 77.58% (460021/592933 zeros)
FINAL SCORE: 0.7533

Saved pruned model for autograder: pruned_model_for_autograder.pt


In [16]:
torch.save(original_model.state_dict(), 'my_model_weights_1.pt', _use_new_zipfile_serialization=False)

In [17]:
torch.save(original_model.state_dict(), 'my_model_weights_2.pt', _use_new_zipfile_serialization=False)

In [18]:
torch.save(original_model.state_dict(), 'my_model_weights_3.pt', _use_new_zipfile_serialization=False)

In [19]:
import glob
import zipfile
import os

pt_files = glob.glob('*.pt')
zip_file_name = 'my_model_weights.zip'

with zipfile.ZipFile(zip_file_name, 'w') as zipf:
    for file in pt_files:
        zipf.write(file, os.path.basename(file))

print(f"Successfully created {zip_file_name} containing: {pt_files}")

Successfully created my_model_weights.zip containing: ['pruned_model_for_autograder.pt', 'my_model_weights_3.pt', 'my_model_weights_2.pt', 'my_model_weights_1.pt']


In [20]:
!ls -lh my_model_weights.zip

-rw-r--r-- 1 root root 9.1M Dec  1 15:14 my_model_weights.zip
